# Learning on Reber-Grammar

Warm up on the Reber-Grammar prediction problem using Keras

## Importing Libraries:

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, TimeDistributed, Masking
from keras.optimizers import RMSprop
from keras.utils import np_utils
import numpy as np
import reber_utility as ru

Using Theano backend.


Defining important parameters:

In [2]:
step_size = 50
n_step=100 #redefine this. IT is to make 1000 reber words

batch_size = 16
n_batch = 12 # total number
n_test_batch = 2 # testing part

train_idxes = np.array(range(0,(n_batch-n_test_batch)*batch_size))
test_idxes = np.array(range((n_batch-n_test_batch)*batch_size, n_batch*batch_size))

N_Epoch = 5

generate and vectorize Data:

In [8]:
m = ru.generate_reber_machine_discrete()

X,Y = m.to_X_and_Y(m.make_words(n_batch*batch_size,min_steps=(step_size*n_step+1)))

In [9]:
print X.shape
print Y.shape

(192, 5000, 6)
(192, 5000, 6)


## Building the model
For the masking layer the value "-1" was chosen since to categorical already uses the values "0" and "1"

In [6]:
model = Sequential()
model.add(Masking(mask_value= -1., batch_input_shape=(batch_size,step_size,len(m.transitions))))
model.add(LSTM(20, return_sequences=True, batch_input_shape=(batch_size,step_size,len(m.transitions)),stateful=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(m.transitions))))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)#learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

## Train model
train model and generate metrics

In [10]:
metrics_train = np.zeros((N_Epoch,len(model.metrics_names)))
metrics_test = np.zeros((N_Epoch,len(model.metrics_names)))

print('Starting to learn:')
for i in range(N_Epoch):
    print('------- {} out of {} Epoch -----'.format(i+1,N_Epoch))

    ## Epochs should take all data; batches presented random, reset at each end of batch_size
    np.random.shuffle(train_idxes)
    batches_idxes = np.reshape(train_idxes, (-1,batch_size))
    for j, batch  in enumerate(batches_idxes):
        print('batch {} of {}'.format(j+1,n_batch-n_test_batch))
        for k in range(n_step):
            metrics_train[i] += model.train_on_batch(X[batch,k*step_size:(k+1)*(step_size)], Y[batch,k*step_size:(k+1)*step_size]) #python 0:3 gives 0,1,2 (which is not intuitive at all)
        model.reset_states() 

    test_batch_idxes = np.reshape(test_idxes,(-1,batch_size))
    for test_batch in test_batch_idxes:
        for k in range(n_step):
            metrics_test[i] += model.test_on_batch(X[test_batch,k*step_size:(k+1)*(step_size)], Y[test_batch,k*step_size:(k+1)*step_size])
        model.reset_states() 

    print('Train results:\t {} \n \t {}'.format(model.metrics_names, metrics_train[i]/(n_batch-n_test_batch) ))
    print('Test results:\t {} \n \t {}'.format(model.metrics_names, metrics_test[i]/(n_test_batch) ))
    
    #TODO: plot graph of accuracy, loss with errorbars dynamicly

Starting to learn:
------- 1 out of 5 Epoch -----
batch 1 of 10
batch 2 of 10
batch 3 of 10
batch 4 of 10
batch 5 of 10
batch 6 of 10
batch 7 of 10
batch 8 of 10
batch 9 of 10
batch 10 of 10
Train results:	 ['loss', 'acc'] 
 	 [ 49.61677154  79.53800003]
Test results:	 ['loss', 'acc'] 
 	 [ 48.85262887  79.43749997]
------- 2 out of 5 Epoch -----
batch 1 of 10
batch 2 of 10
batch 3 of 10
batch 4 of 10
batch 5 of 10
batch 6 of 10
batch 7 of 10
batch 8 of 10
batch 9 of 10
batch 10 of 10
Train results:	 ['loss', 'acc'] 
 	 [ 48.91956176  79.56162502]
Test results:	 ['loss', 'acc'] 
 	 [ 49.03371318  79.45499977]
------- 3 out of 5 Epoch -----
batch 1 of 10
batch 2 of 10
batch 3 of 10
batch 4 of 10
batch 5 of 10
batch 6 of 10
batch 7 of 10
batch 8 of 10
batch 9 of 10
batch 10 of 10
Train results:	 ['loss', 'acc'] 
 	 [ 48.8486317   79.54062497]
Test results:	 ['loss', 'acc'] 
 	 [ 48.82096468  79.4562498 ]
------- 4 out of 5 Epoch -----
batch 1 of 10
batch 2 of 10
batch 3 of 10
batch 4 of 

don't forget to save everything ;)

In [11]:
model.save('embedCerg_model_2.h5')
np.save('embedXdata_2.npy',X)
np.save('embedydata_2.npy',Y)
np.save('embedTrainMetrics_2', metrics_train)
np.save('embedTestMetrics_2', metrics_test)